In [ ]:
import pandas as pd
import numpy as np
import pandas as pd

In [5]:
df_original = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/bronze/df_list_data_news.csv', sep='|')
df_categorized = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/silver/df_categorized_googlenews.csv',
                             sep=';')

df_orginal_reddit = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/bronze/df_list_data_reddit_tratado.csv', sep='|')
df_categorized_reddit = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/silver/df_categorized_reddit.csv', sep=';')


Data cleaning

In [6]:
class DataCleaning:
    def __init__(self):
        self = self

    def standardize(self, df: pd.DataFrame) -> pd.DataFrame:

        df = df.rename(columns={
                            '0':'index',
                            '1':'flg_crime',
                            '2':'cat_crime'
                        })
        
        columns = df.columns

        for column in columns:
            match column:
                case 'flg_crime':
                    df[column] = df[column].str.replace(' ', '')
                    df[column] = df[column].str.replace('|', 'N')
                case 'cat_crime':
                    df[column] = df[column].str.strip()
                    df[column] = df[column].str.upper()
                    df[column] = df[column].str.replace('|', '')
                    df[column] = df[column].str.replace('-', '')
                    df[column] = df[column].str.replace('|', '')
                    df[column] = np.where(df[column].str.contains('CIBERCRIME BANCÁRIO', na=False), df[column].str.extract(r'(CIBERCRIME BANCÁRIO)', expand=False),
                                            df[column])
                    df[column] = np.where(df[column].str.contains('OUTROS CIBERCRIMES', na=False), df[column].str.extract(r'(OUTROS CIBERCRIMES)', expand=False),
                                        df[column])
                    df[column] = np.where(df[column] == 'CIBERCRIMES', 'CIBERCRIME BANCÁRIO',
                                            np.where(df[column].str.contains('CIBERCRIMES \\('), 'OUTROS CIBERCRIMES',
                                                    np.where(df[column].str.contains('OUTROS CIBERCIMES'), 'OUTROS CIBERCRIMES', df[column])))
                    df[column] = np.where((~df[column].str.contains('CIBERCRIME BANCÁRIO')) & (~df[column].str.contains('OUTROS CIBERCRIMES')), '', df[column])
                case 'index':
                    df[column] = df[column].astype(str)
                    df[column] = df[column].str.replace("'", "")
                    df[column] = df[column].str.replace(r'[^0-9]', '', regex=True)
                    df[column] = df[column].astype(int)
        
        return df

In [7]:
data_cleaner = DataCleaning()

# Google News

In [8]:
teste = data_cleaner.standardize(df=df_categorized)

In [9]:
df_concat_google_news = pd.merge(left=df_original,
                     right=teste,
                     left_on='index',
                     right_on='index',
                     how='inner',
                     suffixes=["_original", "_categorized"])\
                .drop_duplicates(subset=['text'])

# Reddit

In [10]:
teste = data_cleaner.standardize(df=df_categorized_reddit)

In [11]:
df_concat_reddit = pd.merge(left=df_orginal_reddit,
                     right=teste,
                     left_on='index',
                     right_on='index',
                     how='inner',
                     suffixes=["_original", "_categorized"])\
                .drop_duplicates(subset=['text'])

# Validação da quantidade dos dois dataframes

In [12]:
df_final = pd.concat([df_concat_reddit, df_concat_google_news])

In [13]:
df_final.count()

index        559
text         559
flg_crime    559
cat_crime    559
dtype: int64

In [14]:
df_final = df_final.drop(['index'], axis=1)

In [226]:
df_final.to_csv(r'C:\Users\jhont\Desktop\g05-ia-seguranca-digital\src\data\processed\gold\df_concat_bigthree.csv', sep='|')

In [15]:
df_concat_reddit.to_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/df_reddit.csv', sep='|')
df_concat_google_news.to_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/df_brave_news.csv', sep='|')


In [ ]:
# Junta os dados classificados pelo profissional de segurança
df_1 = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/df_hllm_annotationV2.csv',
                    sep='|')

df_2 = pd.read_csv(r'/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/df_hllm_annotation2v2.csv',
                    sep='|')

df_concat = pd.concat([df_1, df_2], axis=0, ignore_index=True)
df_concat.rename(columns={'index':'pk'}, inplace=True)

In [42]:
df_concat.to_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/golpebr_classificacoes.csv', sep='|')

-----

Ajusta as colunas de cat_crime e flg_crime. As colunas foram inseridas pelo profissional de segurança para maior controle dos registros, mas não serão utilizadas na base final.

In [43]:
df_text = pd.read_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/df_concat_bigthree.csv', sep='|')
df_text.drop(['flg_crime', 'cat_crime'], axis=1, inplace=True)

In [44]:
df_classificacoes = pd.read_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/golpebr_classificacoes.csv', sep='|')

In [45]:
df_classificacoes['pk'] = df_classificacoes['pk'].astype(int)

In [48]:
df = df_text.merge(
    right=df_classificacoes,
    left_on='index',
    right_on='pk',
    how='inner'
).drop(['Unnamed: 0'], axis=1)

In [51]:
df = df.drop_duplicates(subset=['text'])

In [54]:
df['cibercrime'] = df['cibercrime'].str.replace(' ', '')

In [ ]:
# Aqui exportamos os dados finais, para o dataset final
df.to_csv('/Users/jhonatatirloni/Desktop/g05-ia-seguranca-digital/src/data/processed/gold/golpebr_dataset.csv',
          sep='|')